In [2]:
import os
os.chdir('C:/pytest')

In [3]:
def read_data(filename, encoding = 'cp949'):
    with open(filename, 'r', encoding = encoding) as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

def write_data(data, filename, encoding = 'cp949'):
    with open(filename, 'w', encoding = encoding) as f:
        f.write(data)

data = read_data('ratings.txt', encoding = 'cp949')

In [4]:
from tqdm import tqdm
import rhinoMorph
rn = rhinoMorph.startRhino()

filepath:  c:\project\venv\lib\site-packages
classpath:  c:\project\venv\lib\site-packages\rhinoMorph/lib/rhino.jar
RHINO started!


In [5]:
morphed_data = ''
for data_each in tqdm(data):
    morphed_data_each = rhinoMorph.onlyMorph_list(rn, data_each[1],
    pos = ['NNG', 'NNP', 'VV', 'VA', 'XR', 'IC', 'MM', 'MAG', 'MAJ'])
    joined_data_each = ' '.join(morphed_data_each)
    if joined_data_each:
        morphed_data += data_each[0] + '\t' + joined_data_each + '\t' + data_each[2] + '\n'

write_data(morphed_data, 'ratings_morphed.txt', encoding = 'cp949')

  3%|▎         | 5213/200000 [00:03<02:25, 1343.07it/s]


KeyboardInterrupt: 

In [6]:
data = read_data('ratings_morphed.txt', encoding = 'cp949')
print(len(data))
print(len(data[0]))
print(data[0])

197559
3
['8132799', '디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라 그 어렵 시절 끝 열정 지키 노라노 같 전통 있 같 사람 꿈 꾸 이루 나가 있 감사', '1']


In [ ]:
data_id = [line[0] for line in data]
data_text = [line[1] for line in data]
data_senti = [line[2] for line in data]

positive = read_data('positive.txt')
negative = read_data('negative.txt')

print('positive:', positive)
print('negative:', negative)

In [15]:
def cntWordInLine(data, senti):
    senti_found = []
    for onedata in data:
        oneline_word = onedata.split(' ')
        senti_temp = 0
        for sentiword in senti:
            if sentiword[0] in oneline_word:
                senti_temp += 1
        senti_found.append(senti_temp)
    return senti_found

In [16]:
data_senti_poscnt = cntWordInLine(data_text, positive)
data_senti_negcnt = cntWordInLine(data_text, negative)

print(data_senti_poscnt[:20])
print(data_senti_negcnt[:20])

[5, 1, 0, 0, 2, 1, 0, 0, 0, 1, 1, 1, 0, 1, 2, 0, 1, 0, 1, 0]
[1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1]


In [17]:
import pandas as pd
newdata = pd.DataFrame({'id':data_id, 'text':data_text, 'original':data_senti, 'pos':data_senti_poscnt, 'neg':data_senti_negcnt})
senti_score = newdata['pos'] - newdata['neg']
newdata['senti_score'] = senti_score

newdata.loc[newdata.senti_score > 0, 'new'] = 1
newdata.loc[newdata.senti_score <= 0, 'new'] = 0

newdata.loc[pd.to_numeric(newdata.original) == newdata.new, 'matched'] = 'True'
newdata.loc[pd.to_numeric(newdata.original) != newdata.new, 'matched'] = 'False'

In [12]:
score = newdata.matched.str.count('True').sum() / (newdata.matched.str.count('True').sum() + newdata.matched.str.count('False').sum()) * 100
print(score)

62.60711989835948


In [13]:
newdata.to_csv('newfile.csv', sep=',', encoding = 'cp949', index = False)
newdata.to_csv('newfile2.txt', sep='\t', encoding='cp949', index=False)

In [16]:
import math
def sigmoid(x):
    return 1 / (1 + math.exp(-x))
newdata['sigmoid'] = newdata.senti_score.apply(sigmoid)


In [17]:
newdata.head()

,id,text,original,pos,neg,senti_score,new,matched,sigmoid
0,8132799,디자인 배우 학생 외국 디자이너 일구 전통 통하 발전 문화 산업 부럽 사실 우리나라...,1,5,1,4,1.0,True,0.982014
1,4655635,폴리스스토리 시리즈 뉴 없 최고,1,1,0,1,1.0,True,0.731059
2,9251303,와 연기 진짜 쩔 지루 생각하 몰입 그래 이런 진짜 영화,1,0,1,-1,0.0,False,0.268941
3,10067386,안개 자욱 하 밤하늘 뜨 초승달 같 영화,1,0,0,0,0.0,False,0.500000
4,2190435,사랑 해보 사람 처음 끝 웃 있 영화,1,2,0,2,1.0,True,0.880797
